Мы определяем следующий пул компаний для который будем считать Information Ratio для компаний:
* S&P 500 (^GSPC) - наш бенчмарк
* FORD (F)
* Amazon (AMZN)
* Tesla (TSLA)
* Apple (AAPL)
* Yandex (YNDX)
* NIVIDA(NVDA)
* Airbnb (ABNB)
* CocaCola Consolidated (COKE)
* Netflix (NFLX)


Сначала необходимо определить откуда мы будем брать информацию. Наша команда выбрала сайт Yahoo.com для импорта данных о ценных бумагах компаний, представленных выше.

In [65]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()
import datetime as dt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [55]:
def get_data(name:str):
    '''
    Эта функция импортирует данные по названию компании
    '''
    data = pdr.get_data_yahoo([name], start='2000-1-1', end='2023-11-01')['Adj Close'].reset_index()
    data = pd.DataFrame(data)
    data.rename(columns={'Adj Close':  f'{name}'}, inplace=True)
    return data

short_names = ['^GSPC','F','AMZN','TSLA', 'AAPL', 'YNDX', 'NVDA', 'ABNB', 'COKE', 'NFLX']
data = get_data(short_names[0])
for i in short_names:
    if i != '^GSPC':
        data = pd.merge(data, get_data(i), on = 'Date', how = 'inner')

data



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
0,2020-12-10,3668.100098,7.952764,155.074493,209.023331,121.260773,70.680000,129.442184,144.710007,257.259674,501.089996
1,2020-12-11,3663.459961,7.865564,155.820999,203.330002,120.444107,70.279999,129.851288,139.250000,256.806305,503.220001
2,2020-12-14,3647.489990,7.769641,157.848495,213.276672,119.824234,68.230003,132.799866,130.000000,259.043762,522.419983
3,2020-12-15,3694.620117,7.978924,158.255997,211.083328,125.826248,68.889999,133.316299,124.800003,262.119049,519.780029
4,2020-12-16,3701.169922,7.883004,162.048004,207.589996,125.757378,70.800003,132.138809,137.990005,263.252563,524.830017
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,4186.770020,11.362824,121.389999,212.419998,171.100006,18.940001,417.790009,118.500000,641.820007,411.250000
723,2023-10-26,4137.229980,11.175742,119.570000,205.759995,166.889999,18.940001,403.260010,115.470001,639.960022,403.540009
724,2023-10-27,4117.370117,9.807083,127.739998,207.300003,168.220001,18.940001,405.000000,114.089996,634.080017,397.869995
725,2023-10-30,4166.819824,9.620000,132.710007,197.360001,170.289993,18.940001,411.609985,116.949997,640.890015,410.079987


Теперь необходимо посчитать Return для каждой компании

In [56]:
returns_daily = pd.DataFrame()
returns_daily['Date'] = data['Date']
for company in data.columns[1:]:
    returns_daily[company] = data[company].pct_change()

returns_daily.iloc[1::]

,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
1,2020-12-11,-0.001265,-0.010965,0.004814,-0.027238,-0.006735,-0.005659,0.003161,-0.037731,-0.001762,0.004251
2,2020-12-14,-0.004359,-0.012195,0.013012,0.048919,-0.005147,-0.029169,0.022707,-0.066427,0.008713,0.038154
3,2020-12-15,0.012921,0.026936,0.002582,-0.010284,0.050090,0.009673,0.003889,-0.040000,0.011872,-0.005053
4,2020-12-16,0.001773,-0.012022,0.023961,-0.016550,-0.000547,0.027725,-0.008832,0.105689,0.004324,0.009716
5,2020-12-17,0.005758,0.004425,-0.001506,0.053198,0.006964,-0.002119,0.007457,0.065657,0.005616,0.015376
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,-0.014340,0.013169,-0.055772,-0.018936,-0.013492,0.000000,-0.043149,-0.028768,-0.007141,-0.005994
723,2023-10-26,-0.011833,-0.016464,-0.014993,-0.031353,-0.024606,0.000000,-0.034778,-0.025570,-0.002898,-0.018748
724,2023-10-27,-0.004800,-0.122467,0.068328,0.007484,0.007969,0.000000,0.004315,-0.011951,-0.009188,-0.014051
725,2023-10-30,0.012010,-0.019076,0.038907,-0.047950,0.012305,0.000000,0.016321,0.025068,0.010740,0.030688


Теперь необходимо посчитать Return компаний от каждого года: 2020, 2021, 2022.

In [57]:
returns_2020 = returns_daily[(returns_daily['Date'] >= dt.datetime(2020,12,11)) & (returns_daily['Date'] < dt.datetime(2021,1,4))]
returns_2021 = returns_daily[(returns_daily['Date'] >dt.datetime(2020,12,31)) & (returns_daily['Date'] < dt.datetime(2022,1,1))]
returns_2022 = returns_daily[(returns_daily['Date'] >dt.datetime(2021,12,31)) & (returns_daily['Date'] < dt.datetime(2023,1,1))]
returns_2023 = returns_daily[(returns_daily['Date'] >dt.datetime(2022,12,31)) & (returns_daily['Date'] < dt.datetime(2024,1,1))]
returns_2023

,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
518,2023-01-03,-0.004001,0.004299,0.021667,-0.122422,-0.037405,0.0,-0.020460,-0.007018,-0.069678,0.000237
519,2023-01-04,0.007539,0.028253,-0.007924,0.051249,0.010314,0.0,0.030318,0.044994,0.019595,0.049025
520,2023-01-05,-0.011646,0.019983,-0.023726,-0.029039,-0.010605,0.0,-0.032816,-0.011384,-0.004588,0.000937
521,2023-01-06,0.022841,0.026939,0.035611,0.024651,0.036794,0.0,0.041640,0.009235,0.033549,0.018889
522,2023-01-09,-0.000768,0.008744,0.014870,0.059349,0.004089,0.0,0.051753,0.008134,-0.018980,-0.001204
...,...,...,...,...,...,...,...,...,...,...,...
722,2023-10-25,-0.014340,0.013169,-0.055772,-0.018936,-0.013492,0.0,-0.043149,-0.028768,-0.007141,-0.005994
723,2023-10-26,-0.011833,-0.016464,-0.014993,-0.031353,-0.024606,0.0,-0.034778,-0.025570,-0.002898,-0.018748
724,2023-10-27,-0.004800,-0.122467,0.068328,0.007484,0.007969,0.0,0.004315,-0.011951,-0.009188,-0.014051
725,2023-10-30,0.012010,-0.019076,0.038907,-0.047950,0.012305,0.0,0.016321,0.025068,0.010740,0.030688


In [58]:
def returns(df):
  returns_for_period = dict()
  for company in df.columns[1:]:
    returns_for_period[company] = np.prod(df[company] + 1) - 1
  return returns_for_period



ret_for_2020 = returns(returns_2020)
ret_for_2021 = returns(returns_2021)
ret_for_2022 = returns(returns_2022)
ret_for_2023 = returns(returns_2023)
all_years ={"2020": ret_for_2020, "2021":ret_for_2021, "2022": ret_for_2022, '2023':ret_for_2023}
all_years

{'2020': {'^GSPC': 0.02398243460131666,
  'F': -0.03618398055324368,
  'AMZN': 0.05011786918026462,
  'TSLA': 0.1253448448168213,
  'AAPL': 0.07667964523730242,
  'YNDX': -0.015563079645891431,
  'NVDA': 0.006378900789249631,
  'ABNB': 0.014442652483757756,
  'COKE': 0.02019161589986118,
  'NFLX': 0.0791075144595974},
 '2021': {'^GSPC': 0.2689273866137132,
  'F': 1.374810011675513,
  'AMZN': 0.023767773568754258,
  'TSLA': 0.49755559240135483,
  'AAPL': 0.3464821012316637,
  'YNDX': -0.1304972922119435,
  'NVDA': 1.2548208811376416,
  'ABNB': 0.1341280792376034,
  'COKE': 1.3324436535490651,
  'NFLX': 0.11412354447068251},
 '2022': {'^GSPC': -0.1944282720342928,
  'F': -0.4218023480304496,
  'AMZN': -0.4961521476592684,
  'TSLA': -0.6503151169866451,
  'AAPL': -0.2640419931559316,
  'YNDX': -0.686942139932932,
  'NVDA': -0.5026464500820724,
  'ABNB': -0.4864556599254213,
  'COKE': -0.17089098671914027,
  'NFLX': -0.5105238634755285},
 '2023': {'^GSPC': 0.09227758944849596,
  'F': -0.07

In [59]:
returns_for_all_per = returns(returns_daily)
returns_for_all_per

{'^GSPC': 0.14331661978558108,
 'F': 0.22598891939719756,
 'AMZN': -0.14176733121701957,
 'TSLA': -0.03915033945556223,
 'AAPL': 0.40828728419697047,
 'YNDX': -0.7320316857345757,
 'NVDA': 2.150441175967857,
 'ABNB': -0.18257207223118688,
 'COKE': 1.4738038537892257,
 'NFLX': -0.17841105300414128}

Посчитаем Tracking Error для каждой компании за каждый год:

In [60]:
def tracking_error(df):
  tracking_errors = dict()
  diff_returns = pd.DataFrame()
  # Сначала необходимо посчитать разницу в Return бенчмарка и Return компаний
  for company in df.columns[2:]:
    diff_returns[f'{company}_diff'] = df[company] - df['^GSPC']
  for company in diff_returns.columns[2:]:
    tracking_errors[company[:4]] = diff_returns[company].std() * np.sqrt(252)
  return tracking_errors

err_2020 = tracking_error(returns_2020)
err_2021 = tracking_error(returns_2021)
err_2022= tracking_error(returns_2022)
err_2023 = tracking_error(returns_2023)
all_err = {"2020": err_2020, "2021":err_2021, "2022": err_2022, '2023':err_2023}
all_err

{'2020': {'TSLA': 0.5597277546244945,
  'AAPL': 0.2713988973475051,
  'YNDX': 0.2515552807320569,
  'NVDA': 0.203595417674456,
  'ABNB': 0.8076492882584391,
  'COKE': 0.18051879445753968,
  'NFLX': 0.28620657652741816},
 '2021': {'TSLA': 0.5026382749344234,
  'AAPL': 0.1872388576592721,
  'YNDX': 0.29072922429512427,
  'NVDA': 0.38810862763179593,
  'ABNB': 0.5110617437881809,
  'COKE': 0.31666568893335884,
  'NFLX': 0.29133395646312293},
 '2022': {'TSLA': 0.5461104766236966,
  'AAPL': 0.18168077709895927,
  'YNDX': 0.5764934647687351,
  'NVDA': 0.4469453223364297,
  'ABNB': 0.46977631441120304,
  'COKE': 0.43573520599878984,
  'NFLX': 0.596260921528842},
 '2023': {'TSLA': 0.5028394787600811,
  'AAPL': 0.14639728761854814,
  'YNDX': 0.13462511268990465,
  'NVDA': 0.45344988335586617,
  'ABNB': 0.3776402094435877,
  'COKE': 0.332607698450412,
  'NFLX': 0.35898510664531746}}

Сейчас необходимо посчитать Tracking Error для каждой компании за весь период:

In [61]:
tracking_errors = tracking_error(returns_daily)
tracking_errors

{'TSLA': 0.5194602653581324,
 'AAPL': 0.17635551382801162,
 'YNDX': 0.3879567259781441,
 'NVDA': 0.4270905870938643,
 'ABNB': 0.46841376848104527,
 'COKE': 0.3643264410716421,
 'NFLX': 0.43675139988463957}

Теперь считаем сам Information Ratio для каждой компании по годам

In [78]:
def IR(df):
  information_ratio = dict()
  for company in tracking_errors.keys():
    information_ratio[company] = (df[company] - df['^GSPC']) / tracking_errors[company]
  return information_ratio

ratio_for_2020 = pd.DataFrame(IR(all_years['2020']), index = [0])
ratio_for_2021 = pd.DataFrame(IR(all_years['2021']), index = [0])
ratio_for_2022 = pd.DataFrame(IR(all_years['2022']), index = [0])
ratio_for_2023 = pd.DataFrame(IR(all_years['2023']), index = [0])
print('for 2020: \n' +  f"{ratio_for_2020} \n")
print('for 2021: \n' + f"{ratio_for_2021}\n")
print('for 2022: \n' + f"{ratio_for_2022}\n")
print('for 2023: \n' + f"{ratio_for_2023}\n")

for 2020: 
      TSLA      AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0  0.19513  0.298812 -0.101933 -0.041217 -0.020366 -0.010405  0.126216 

for 2021: 
       TSLA      AAPL     YNDX      NVDA      ABNB     COKE      NFLX
0  0.440126  0.439763 -1.02956  2.308394 -0.287778  2.91913 -0.354444

for 2022: 
       TSLA      AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0 -0.877616 -0.394735 -1.269507 -0.721669 -0.623439  0.064605 -0.723743

for 2023: 
      TSLA      AAPL      YNDX      NVDA      ABNB      COKE      NFLX
0  1.03604  1.290876 -0.237855  3.978524  0.621739  0.443602  0.695704



Теперь считаем сам Information Ratio для каждой компании за весь период

In [63]:
ratio_for_all = IR(returns_for_all_per)
ratio_for_all

{'TSLA': -0.35126259198159226,
 'AAPL': 1.502480181423749,
 'YNDX': -2.2563039816184816,
 'NVDA': 4.699528898165948,
 'ABNB': -0.6957282512714084,
 'COKE': 3.6519096173478505,
 'NFLX': -0.7366379887384477}

Графики

In [81]:
fig = make_subplots(rows=2, cols=1)
data = pd.concat([ratio_for_2020, ratio_for_2021, ratio_for_2022, ratio_for_2023], ignore_index=True)

fig.add_trace(go.Scatter( y=data.iloc[0]), row=1, col=1)
# fig.add_trace(go.Bar(y=[2, 1, 3]), row=2, col=1)

fig.show()